In [ ]:
import streamlit as st
import streamlit_authenticator as stauth
from cryptography.fernet import fernet
import json
import os

#Function to generate and load encryption key
def load_key():
    if os.path.exists("secret.key"):
       with open("secret.key","rb") as key_file:
          return key_file.read()
    else:
        key = Fernet.generate_key()
        with open("secret.key", "wb") as key_file:
            key_file.write(key)
        return key

# Function to encrypt data
def encrypt_data(data, key):
    fernet = Fernet(key)
    encrypted = fernet.encrypt(data.encode())
    return encrypted

# Function to decrypt data
def decrypt_data(encrypted_data, key):
    fernet = Fernet(key)
    decrypted = fernet.decrypt(encrypted_data).decode()
    return decrypted

# Load encryption key
key = load_key()

# User authentication setup
names = ["Alice", "Bob"]
usernames = ["alice", "bob"]
passwords = ["password123", "password456"]
hashed_passwords = stauth.Hasher(passwords).generate()

authenticator = stauth.Authenticate(
    {"credentials": {"usernames": {usernames[i]: {"name": names[i], "password": hashed_passwords[i]} for i in range(len(names))}}},
    "cookie_name",
    "signature_key",
    cookie_expiry_days=30,
)

# Streamlit UI
st.title("Secure Data Encryption System")

# Login
name, authentication_status, username = authenticator.login("Login", "main")

if authentication_status:
    authenticator.logout("Logout", "main")
    st.write(f"Welcome {name}!")

    # Data storage (simulated with a dictionary)
    user_data = {}

    # Input for new data
    new_data = st.text_area("Enter data to encrypt:")

    if new_data:
        encrypted = encrypt_data(new_data, key)
        user_data[username] = encrypted
        st.write("Data encrypted and stored.")

    # Display stored data
    if username in user_data:
        decrypted = decrypt_data(user_data[username], key)
        st.write("Stored data:", decrypted)

elif authentication_status is False:
    st.error("Username/password is incorrect")
elif authentication_status is None:
    st.warning("Please enter your username and password")